# Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from datetime import datetime
import os
import sys

import mujoco
import mediapy as media
from gym import Env
from gym.spaces import Discrete, Box

import stable_baselines3
from stable_baselines3.common.env_checker import check_env
import torch as th
from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.logger import configure
from stable_baselines3.common.logger import Logger, CSVOutputFormat, HumanOutputFormat

# Cargar los datos de captura de movimiento y simulación

In [2]:
datos = pd.read_csv('datos_seguimiento_3_polyfit15_conpos.csv',sep=",")
array_datos = datos.to_numpy() #guarda los datos del csv cargado como arreglo de numpy

with open("mujoco64.xml", "r", encoding="utf-8") as f:
    xml = f.read() #Leer el archivo de mujoco cargado al entorno de ejecución y lo guarda en la variable xml


# Gym Enviroment

In [3]:
class RobotBipedoEnv(Env):
  def __init__(self, xml_text, tracking_array, tiempo_max_sim, tracking_fps=30):

    #Algunos parámetros asociados a la física del robot y la simulación de MuJoCo
    self.simulation_time_step=0.002 #La simulación corre en pasos de 0.002 segundos (frecuencia de 500 Hz)
    self.motor_velocity_deg_s=90 #La velocidad de los actuadores máxima es 90 deg/s **se modifica por reducciones
    self.motor_velocity_rad_s=(self.motor_velocity_deg_s)*(np.pi/180) #Conversión a rad/s
    self.max_step_motors=self.motor_velocity_rad_s*self.simulation_time_step #Traducción a máximo giro en radianes


    #Se crea la simulación con el xml cargado
    # Make model and data
    self.model = mujoco.MjModel.from_xml_string(xml_text)
    self.data = mujoco.MjData(self.model)


    # El estado es definido por 19 valores, donde se almacenan las coordenadas en 3 dimensiones de los 6 tracking points y el tiempo
    # Orden: Cadera pierna 1, Rodilla pierna 1, Tobillo pierna 1, Cadera pierna 2, Rodilla pierna 2, Tobillo pierna 2, tiempo
    # 0  Cadera pierna 1 x
    # 1  Cadera pierna 1 y
    # 2  Cadera pierna 1 z
    # 3  Rodilla pierna 1 x
    # 4  Rodilla pierna 1 y
    # 5  Rodilla pierna 1 z
    # 6  Tobillo pierna 1 x
    # 7  Tobillo pierna 1 y
    # 8  Tobillo pierna 1 z
    # 9  Cadera pierna 2 x
    # 10 Cadera pierna 2 y
    # 11 Cadera pierna 2 z
    # 12 Rodilla pierna 2 x
    # 13 Rodilla pierna 2 y
    # 14 Rodilla pierna 2 z
    # 15 Tobillo pierna 2 x
    # 16 Tobillo pierna 2 y
    # 17 Tobillo pierna 2 z
    # 18 tiempo
    self.observation_space = Box(low=-np.inf, high=np.inf, shape=(19, ))

    # Las acciones son definidas como un continuo en cada paso de la simulación. Limitado por el paso de la simulación y velocidad motores
    # Orden: Motor1_cadera_pierna1,Motor2_cadera_pierna1,Motor_rodilla_pierna1,Motor_tobillo_pierna1,Motor1_cadera_pierna2,Motor2_cadera_pierna2,Motor_rodilla_pierna2,Motor_tobillo_pierna2
    self.action_space = Box(low=-1, high=1, shape=(8,))

    #El estado inicial está dado por el tracking inicial
    self.state=np.concatenate((self.data.geom('TrackerCadera_pierna1_geom').xpos,
                              self.data.geom('TrackerRodilla_pierna1_geom').xpos,
                              self.data.geom('TrackerTobillo_pierna1_geom').xpos,
                              self.data.geom('TrackerCadera_pierna2_geom').xpos,
                              self.data.geom('TrackerRodilla_pierna2_geom').xpos,
                              self.data.geom('TrackerTobillo_pierna2_geom').xpos,
                              np.array([self.data.time])))

    #Se configura un tiempo final de simulación
    if(tiempo_max_sim<=4.9):
      self.final_simulation_time=tiempo_max_sim
    else:
      self.final_simulation_time=4.9  #la simulación se termina en 5 segundos

    #Se almacena la secuencia de tracking de movimiento a seguir
    #Orden por columnas: Número de frame a X Hz, Ángulo cadera-rodilla_p1, Ángulo rodilla-tobillo_p1, Ángulo cadera-tobillo_p1, Ángulo cadera-rodilla_p2, Ángulo rodilla-tobillo_p2, Ángulo cadera-tobillo_p2 
    self.framerate=tracking_fps
    self.tracking=tracking_array

    #Crea una lista vacía para guardar frames de simulación (60 Hz) y otras opciones
    self.height=480
    self.width=640
    self.frames=[]
    self.renderer = mujoco.Renderer(self.model, self.height, self.width)

    #Inicializa el primer objetivo
    self.obj_frame=1
    self.objective_pos=-self.tracking[self.obj_frame-1][7]


  #Función donde se accionan los controladores, según la acción y los límites
  def actuate_model(self, action):
    #Añade a los controladores los ángulos del vector de acciones
    self.data.actuator("motor1_cadera_pierna1").ctrl+=np.clip(action[0],-self.max_step_motors,self.max_step_motors)
    self.data.actuator("motor2_cadera_pierna1").ctrl+=np.clip(action[1],-self.max_step_motors,self.max_step_motors)
    self.data.actuator("motor_rodilla_pierna1").ctrl+=np.clip(action[2],-self.max_step_motors,self.max_step_motors)
    self.data.actuator("motor_tobillo_pierna1").ctrl+=np.clip(action[3],-self.max_step_motors,self.max_step_motors)
    self.data.actuator("motor1_cadera_pierna2").ctrl+=np.clip(action[4],-self.max_step_motors,self.max_step_motors)
    self.data.actuator("motor2_cadera_pierna2").ctrl+=np.clip(action[5],-self.max_step_motors,self.max_step_motors)
    self.data.actuator("motor_rodilla_pierna2").ctrl+=np.clip(action[6],-self.max_step_motors,self.max_step_motors)
    self.data.actuator("motor_tobillo_pierna2").ctrl+=np.clip(action[7],-self.max_step_motors,self.max_step_motors)

    #self.data.actuator("arnes1").ctrl=-self.objective_pos+0.05
    #self.data.actuator("arnes2").ctrl=-self.objective_pos+0.05
    #self.data.actuator("arnes3").ctrl=-self.objective_pos+0.05
    
    #Usa clip para aplicar los límites superiores e inferiores
    self.data.actuator("motor1_cadera_pierna1").ctrl=np.clip(self.data.actuator("motor1_cadera_pierna1").ctrl,-0.95,0.698)
    self.data.actuator("motor2_cadera_pierna1").ctrl=np.clip(self.data.actuator("motor2_cadera_pierna1").ctrl,-0.698,0.95)
    self.data.actuator("motor_rodilla_pierna1").ctrl=np.clip(self.data.actuator("motor_rodilla_pierna1").ctrl,0,1.48)
    self.data.actuator("motor_tobillo_pierna1").ctrl=np.clip(self.data.actuator("motor_tobillo_pierna1").ctrl,-0.31,0.61)
    self.data.actuator("motor1_cadera_pierna2").ctrl=np.clip(self.data.actuator("motor1_cadera_pierna2").ctrl,-0.95,0.698)
    self.data.actuator("motor2_cadera_pierna2").ctrl=np.clip(self.data.actuator("motor2_cadera_pierna2").ctrl,-0.698,0.95)
    self.data.actuator("motor_rodilla_pierna2").ctrl=np.clip(self.data.actuator("motor_rodilla_pierna2").ctrl,0,1.48)
    self.data.actuator("motor_tobillo_pierna2").ctrl=np.clip(self.data.actuator("motor_tobillo_pierna2").ctrl,-0.31,0.61)

    #self.data.actuator("arnes1").ctrl=np.clip(self.data.actuator("arnes1").ctrl,-1,2)
    #self.data.actuator("arnes2").ctrl=np.clip(self.data.actuator("arnes2").ctrl,-1,2)
    #self.data.actuator("arnes3").ctrl=np.clip(self.data.actuator("arnes3").ctrl,-1,2)
    


  def step(self,action):

    #Ejecuta la acción y se cambian los controladores en la simulación
    self.actuate_model(action)
    #Da el paso de simulación
    mujoco.mj_step(self.model, self.data)
    #Lee las nuevas posiciones (estado)
    self.state=np.concatenate((self.data.geom('TrackerCadera_pierna1_geom').xpos,
                              self.data.geom('TrackerRodilla_pierna1_geom').xpos,
                              self.data.geom('TrackerTobillo_pierna1_geom').xpos,
                              self.data.geom('TrackerCadera_pierna2_geom').xpos,
                              self.data.geom('TrackerRodilla_pierna2_geom').xpos,
                              self.data.geom('TrackerTobillo_pierna2_geom').xpos,
                              np.array([self.data.time])))

    #Calcula una reward, tiempo de simulación y objetivo de tracking de movimiento en frame específico
    #PENDIENTE: implementar cálculo de cuaterniones
    #PENDIENTE: implementar velocidad, end effector, center of mass
    #calcula los ángulos entre los puntos de tracking
    # 0  Cadera pierna 1 x
    # 1  Cadera pierna 1 y
    # 2  Cadera pierna 1 z
    # 3  Rodilla pierna 1 x
    # 4  Rodilla pierna 1 y
    # 5  Rodilla pierna 1 z
    # 6  Tobillo pierna 1 x
    # 7  Tobillo pierna 1 y
    # 8  Tobillo pierna 1 z
    # 9  Cadera pierna 2 x
    # 10 Cadera pierna 2 y
    # 11 Cadera pierna 2 z
    # 12 Rodilla pierna 2 x
    # 13 Rodilla pierna 2 y
    # 14 Rodilla pierna 2 z
    # 15 Tobillo pierna 2 x
    # 16 Tobillo pierna 2 y
    # 17 Tobillo pierna 2 z
    #*se multiplica por menos porque el eje coordenado y en MuJoCo está invertido con respecto al tracking
    current_angle_cadera_rodilla_pierna1=np.arctan((-(self.state[4]-self.state[1]))/np.abs((self.state[5]-self.state[2])))
    current_angle_rodilla_tobillo_pierna1=np.arctan((-(self.state[7]-self.state[4]))/np.abs((self.state[8]-self.state[5])))
    current_angle_cadera_tobillo_pierna1=np.arctan((-(self.state[7]-self.state[1]))/np.abs((self.state[8]-self.state[2])))
    current_angle_cadera_rodilla_pierna2=np.arctan((-(self.state[13]-self.state[10]))/np.abs((self.state[14]-self.state[11])))
    current_angle_rodilla_tobillo_pierna2=np.arctan((-(self.state[16]-self.state[13]))/np.abs((self.state[17]-self.state[14])))
    current_angle_cadera_tobillo_pierna2=np.arctan((-(self.state[16]-self.state[10]))/np.abs((self.state[17]-self.state[11])))
    current_pos=self.state[1]
    #revisa el tiempo actual de simulación, contraste con los objetivos de la captura de movimiento, y revisa los ángulos objetivo
    if(self.data.time > self.obj_frame/self.framerate):
      self.obj_frame+=1
    objective_angle_cadera_rodilla_pierna1=self.tracking[self.obj_frame-1][1]
    objective_angle_rodilla_tobillo_pierna1=self.tracking[self.obj_frame-1][2]
    objective_angle_cadera_tobillo_pierna1=self.tracking[self.obj_frame-1][3]
    objective_angle_cadera_rodilla_pierna2=self.tracking[self.obj_frame-1][4]
    objective_angle_rodilla_tobillo_pierna2=self.tracking[self.obj_frame-1][5]
    objective_angle_cadera_tobillo_pierna2=self.tracking[self.obj_frame-1][6]
    self.objective_pos=-self.tracking[self.obj_frame-1][7]
    #calcula reward
    rp=np.exp(-2*((objective_angle_cadera_rodilla_pierna1-current_angle_cadera_rodilla_pierna1)**2+(objective_angle_rodilla_tobillo_pierna1-current_angle_rodilla_tobillo_pierna1)**2+(objective_angle_cadera_rodilla_pierna2-current_angle_cadera_rodilla_pierna2)**2+(objective_angle_rodilla_tobillo_pierna2-current_angle_rodilla_tobillo_pierna2)**2))
    rv=0
    re=np.exp(-40*((objective_angle_cadera_tobillo_pierna1-current_angle_cadera_tobillo_pierna1)**2+(objective_angle_cadera_tobillo_pierna2-current_angle_cadera_tobillo_pierna2)**2))
    rc=np.exp(-10*((self.objective_pos-current_pos)**2))
    reward=0.65*rp+0.1*rv+0.15*re+0.1*rc

    #Condiciones de finalización
    #si pierde el equilibrio y se cae
    done=False
    if(self.state[2]<-0.15 or self.state[11]<-0.15):
      done=True
      reward=-80
    if(self.data.time >= self.final_simulation_time):
      done=True

    #Complementar
    info={
      "reward": reward,
      "state": self.state,
      "objective": self.tracking[self.obj_frame-1],
      "time": self.data.time
    }

    #Retornos
    return self.state, reward, done, info


  def render(self):
    #después de la función step(action) se llama a esta función, así que se comentan algunas líneas
    if len(self.frames) < self.data.time * self.framerate:
      self.renderer.update_scene(self.data, camera='camara_rodilla')
      pixels = self.renderer.render()
      self.frames.append(pixels)

  def reset(self):
    self.frames=[] #borra los frames
    self.obj_frame=1 #reinicia el objetivo de tracking
    mujoco.mj_resetData(self.model, self.data) #reinicia la simulación
    self.state=np.concatenate((self.data.geom('TrackerCadera_pierna1_geom').xpos,
                              self.data.geom('TrackerRodilla_pierna1_geom').xpos,
                              self.data.geom('TrackerTobillo_pierna1_geom').xpos,
                              self.data.geom('TrackerCadera_pierna2_geom').xpos,
                              self.data.geom('TrackerRodilla_pierna2_geom').xpos,
                              self.data.geom('TrackerTobillo_pierna2_geom').xpos,
                              np.array([self.data.time]))) #cambia el estado tras el reinicio
    self.objective_pos=-self.tracking[self.obj_frame-1][7]
    return self.state


# Creación del ambiente

In [4]:
env=RobotBipedoEnv(xml_text=xml, tracking_array=array_datos, tiempo_max_sim=2, tracking_fps=30)
#el ambiente es creado usando la variable xml donde se cargó la simulación y el arreglo con la captura de movimiento

# Entrenamiento con Stable Baselines 3

In [5]:
policy_kwargs = dict(activation_fn=th.nn.ReLU,net_arch=dict(pi=[1024,512,256,128], vf=[1024,512,256,128]))
model = PPO(
        "MlpPolicy", env,
        learning_rate=0.0001,
        n_steps=4096,
        batch_size=1024,
        n_epochs=10,
        gamma=0.99,
        gae_lambda=0.99,
        clip_range=0.15,
        ent_coef=0.001,
        policy_kwargs=policy_kwargs,
        verbose=0
    )

c:\Users\ssola_txrzbqq\AppData\Local\Programs\Python\Python38\lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(
c:\Users\ssola_txrzbqq\AppData\Local\Programs\Python\Python38\lib\site-packages\stable_baselines3\common\on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(

In [6]:
# Crear el nombre del archivo CSV con timestamp
timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M')
csv_path = f"./logs/ppo_m65_{timestamp}.csv"

# Crear un nuevo logger con CSV y salida en pantalla
new_logger = Logger(
    folder="./logs/",
    output_formats=[
        CSVOutputFormat(csv_path),  # Guardar en CSV con timestamp
        HumanOutputFormat(sys.stdout)  # Mostrar en el notebook
    ]
)

# Asignar el nuevo logger al modelo
model.set_logger(new_logger)

model.learn(total_timesteps=5_000_000, progress_bar=True)

Output()

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1e+03    |
|    ep_rew_mean     | 322      |
| time/              |          |
|    fps             | 316      |
|    iterations      | 1        |
|    time_elapsed    | 12       |
|    total_timesteps | 4096     |
---------------------------------


------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 1e+03        |
|    ep_rew_mean          | 323          |
| time/                   |              |
|    fps                  | 322          |
|    iterations           | 2            |
|    time_elapsed         | 25           |
|    total_timesteps      | 8192         |
| train/                  |              |
|    approx_kl            | 7.132228e-05 |
|    clip_fraction        | 0            |
|    clip_range           | 0.15         |
|    entropy_loss         | -11.4        |
|    explained_variance   | -0.000642    |
|    learning_rate        | 0.0001       |
|    loss                 | 111          |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.000274    |
|    std                  | 1            |
|    value_loss           | 242          |
------------------------------------------


-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 1e+03         |
|    ep_rew_mean          | 319           |
| time/                   |               |
|    fps                  | 326           |
|    iterations           | 3             |
|    time_elapsed         | 37            |
|    total_timesteps      | 12288         |
| train/                  |               |
|    approx_kl            | 2.5293019e-05 |
|    clip_fraction        | 0             |
|    clip_range           | 0.15          |
|    entropy_loss         | -11.3         |
|    explained_variance   | -0.129        |
|    learning_rate        | 0.0001        |
|    loss                 | 70.4          |
|    n_updates            | 20            |
|    policy_gradient_loss | -0.000221     |
|    std                  | 0.999         |
|    value_loss           | 183           |
-------------------------------------------


-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 1e+03         |
|    ep_rew_mean          | 317           |
| time/                   |               |
|    fps                  | 329           |
|    iterations           | 4             |
|    time_elapsed         | 49            |
|    total_timesteps      | 16384         |
| train/                  |               |
|    approx_kl            | 5.1196315e-05 |
|    clip_fraction        | 0             |
|    clip_range           | 0.15          |
|    entropy_loss         | -11.3         |
|    explained_variance   | -0.5          |
|    learning_rate        | 0.0001        |
|    loss                 | 38.7          |
|    n_updates            | 30            |
|    policy_gradient_loss | -0.000256     |
|    std                  | 0.999         |
|    value_loss           | 105           |
-------------------------------------------


...

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 1e+03        |
|    ep_rew_mean          | 584          |
| time/                   |              |
|    fps                  | 356          |
|    iterations           | 1217         |
|    time_elapsed         | 13979        |
|    total_timesteps      | 4984832      |
| train/                  |              |
|    approx_kl            | 0.0036083104 |
|    clip_fraction        | 0.0705       |
|    clip_range           | 0.15         |
|    entropy_loss         | -11          |
|    explained_variance   | 0.998        |
|    learning_rate        | 0.0001       |
|    loss                 | 0.345        |
|    n_updates            | 12160        |
|    policy_gradient_loss | -0.00218     |
|    std                  | 0.96         |
|    value_loss           | 0.919        |
------------------------------------------


------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 1e+03        |
|    ep_rew_mean          | 585          |
| time/                   |              |
|    fps                  | 356          |
|    iterations           | 1218         |
|    time_elapsed         | 13990        |
|    total_timesteps      | 4988928      |
| train/                  |              |
|    approx_kl            | 0.0057059927 |
|    clip_fraction        | 0.0988       |
|    clip_range           | 0.15         |
|    entropy_loss         | -11          |
|    explained_variance   | 0.998        |
|    learning_rate        | 0.0001       |
|    loss                 | 0.442        |
|    n_updates            | 12170        |
|    policy_gradient_loss | -0.00369     |
|    std                  | 0.959        |
|    value_loss           | 0.806        |
------------------------------------------


------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 1e+03        |
|    ep_rew_mean          | 586          |
| time/                   |              |
|    fps                  | 356          |
|    iterations           | 1219         |
|    time_elapsed         | 14001        |
|    total_timesteps      | 4993024      |
| train/                  |              |
|    approx_kl            | 0.0033274281 |
|    clip_fraction        | 0.0698       |
|    clip_range           | 0.15         |
|    entropy_loss         | -11          |
|    explained_variance   | 0.998        |
|    learning_rate        | 0.0001       |
|    loss                 | 0.223        |
|    n_updates            | 12180        |
|    policy_gradient_loss | -0.00258     |
|    std                  | 0.959        |
|    value_loss           | 0.557        |
------------------------------------------


------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 1e+03        |
|    ep_rew_mean          | 587          |
| time/                   |              |
|    fps                  | 356          |
|    iterations           | 1220         |
|    time_elapsed         | 14013        |
|    total_timesteps      | 4997120      |
| train/                  |              |
|    approx_kl            | 0.0056760446 |
|    clip_fraction        | 0.0586       |
|    clip_range           | 0.15         |
|    entropy_loss         | -11          |
|    explained_variance   | 0.999        |
|    learning_rate        | 0.0001       |
|    loss                 | 0.255        |
|    n_updates            | 12190        |
|    policy_gradient_loss | -0.00209     |
|    std                  | 0.96         |
|    value_loss           | 0.826        |
------------------------------------------


------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 1e+03        |
|    ep_rew_mean          | 587          |
| time/                   |              |
|    fps                  | 356          |
|    iterations           | 1221         |
|    time_elapsed         | 14024        |
|    total_timesteps      | 5001216      |
| train/                  |              |
|    approx_kl            | 0.0065431055 |
|    clip_fraction        | 0.12         |
|    clip_range           | 0.15         |
|    entropy_loss         | -11          |
|    explained_variance   | 0.998        |
|    learning_rate        | 0.0001       |
|    loss                 | 0.37         |
|    n_updates            | 12200        |
|    policy_gradient_loss | -0.00477     |
|    std                  | 0.961        |
|    value_loss           | 0.823        |
------------------------------------------


In [7]:
model.save(f"ppo_m65_{timestamp}")

In [8]:
eval_env=RobotBipedoEnv(xml_text=xml, tracking_array=array_datos, tiempo_max_sim=2)
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=1)
print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

c:\Users\ssola_txrzbqq\AppData\Local\Programs\Python\Python38\lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


mean_reward:453.61 +/- 0.00


In [9]:
episodios=1
for i in range(0,episodios):
  state=eval_env.reset()
  new_state=state
  done=False
  recompensa_acumulada=0
  pasos=0

  while not done:
    state=new_state
    pasos+=1
    action, _states = model.predict(state, deterministic=True)
    new_state,reward,done,info=eval_env.step(action)
    recompensa_acumulada+=reward
  print("Episodio {}: Recompensa total {} en {} pasos de tiempo.".format(i+1,recompensa_acumulada,pasos))

Episodio 1: Recompensa total 453.61472183843824 en 1000 pasos de tiempo.


In [10]:
state=eval_env.reset()
new_state=state
done=False
recompensa_acumulada=0
pasos=0

while not done:
  state=new_state
  pasos+=1
  action, _states = model.predict(state, deterministic=True)
  #print(action)
  new_state,reward,done,info=eval_env.step(action)
  print(new_state[1])
  recompensa_acumulada+=reward
  print(reward)
  eval_env.render()

print("Recompensa total {} en {} pasos de tiempo.".format(recompensa_acumulada,pasos))
media.show_video(eval_env.frames, fps=eval_env.framerate)

0.004540999773179909
0.8334555399406508
0.004540999773179909
0.8334311669736204
0.004540999773179909
0.8334493844574296
0.004540999773179909
0.8335576222490496
0.004540999773179909
0.833784723571193
0.004540999773179909
0.8341438772881755
0.004540999773179909
0.8346315967729624
0.004540999773179909
0.8352301712888991
0.004540999773179909
0.8359106283849739
0.004540999773179909
0.8366349287972209
0.004540999773179909
0.8373521153099676
0.004540999773179909
0.8379846423468998
0.004540999773179909
0.8383304119737913
0.004540999773179909
0.8383473877103472
0.004540999773179909
0.8382608135668358
0.004540999773179909
0.8381944074599316
0.004540999773179909
0.8219703811900263
0.004540999773179909
0.821968761397863
0.004540999773179909
0.8219919699002382
0.004540999773179909
0.8220165555664066
0.004540999773179909
0.8220209092903031
0.004540999773179909
0.821999611781105
0.004540999773179909
0.8219699276661957
0.004540999773179909
0.8219474735115891
0.004540999773179909
0.8219381014609488
0.0

This browser does not support the video tag.


In [ ]:
class RobotBipedoEnv2(Env):
  def __init__(self, xml_text, tracking_array, tiempo_max_sim, tracking_fps=30):

    #Algunos parámetros asociados a la física del robot y la simulación de MuJoCo
    self.simulation_time_step=0.002 #La simulación corre en pasos de 0.002 segundos (frecuencia de 500 Hz)
    self.motor_velocity_deg_s=90 #La velocidad de los actuadores máxima es 90 deg/s **se modifica por reducciones
    self.motor_velocity_rad_s=(self.motor_velocity_deg_s)*(np.pi/180) #Conversión a rad/s
    self.max_step_motors=self.motor_velocity_rad_s*self.simulation_time_step #Traducción a máximo giro en radianes


    #Se crea la simulación con el xml cargado
    # Make model and data
    self.model = mujoco.MjModel.from_xml_string(xml_text)
    self.data = mujoco.MjData(self.model)


    # El estado es definido por 19 valores, donde se almacenan las coordenadas en 3 dimensiones de los 6 tracking points y el tiempo
    # Orden: Cadera pierna 1, Rodilla pierna 1, Tobillo pierna 1, Cadera pierna 2, Rodilla pierna 2, Tobillo pierna 2, tiempo
    # 0  Cadera pierna 1 x
    # 1  Cadera pierna 1 y
    # 2  Cadera pierna 1 z
    # 3  Rodilla pierna 1 x
    # 4  Rodilla pierna 1 y
    # 5  Rodilla pierna 1 z
    # 6  Tobillo pierna 1 x
    # 7  Tobillo pierna 1 y
    # 8  Tobillo pierna 1 z
    # 9  Cadera pierna 2 x
    # 10 Cadera pierna 2 y
    # 11 Cadera pierna 2 z
    # 12 Rodilla pierna 2 x
    # 13 Rodilla pierna 2 y
    # 14 Rodilla pierna 2 z
    # 15 Tobillo pierna 2 x
    # 16 Tobillo pierna 2 y
    # 17 Tobillo pierna 2 z
    # 18 tiempo
    self.observation_space = Box(low=-np.inf, high=np.inf, shape=(19, ))

    # Las acciones son definidas como un continuo en cada paso de la simulación. Limitado por el paso de la simulación y velocidad motores
    # Orden: Motor1_cadera_pierna1,Motor2_cadera_pierna1,Motor_rodilla_pierna1,Motor_tobillo_pierna1,Motor1_cadera_pierna2,Motor2_cadera_pierna2,Motor_rodilla_pierna2,Motor_tobillo_pierna2
    self.action_space = Box(low=-1, high=1, shape=(8,))

    #El estado inicial está dado por el tracking inicial
    self.state=np.concatenate((self.data.geom('TrackerCadera_pierna1_geom').xpos,
                              self.data.geom('TrackerRodilla_pierna1_geom').xpos,
                              self.data.geom('TrackerTobillo_pierna1_geom').xpos,
                              self.data.geom('TrackerCadera_pierna2_geom').xpos,
                              self.data.geom('TrackerRodilla_pierna2_geom').xpos,
                              self.data.geom('TrackerTobillo_pierna2_geom').xpos,
                              np.array([self.data.time])))

    #Se configura un tiempo final de simulación
    if(tiempo_max_sim<=4.9):
      self.final_simulation_time=tiempo_max_sim
    else:
      self.final_simulation_time=4.9  #la simulación se termina en 5 segundos

    #Se almacena la secuencia de tracking de movimiento a seguir
    #Orden por columnas: Número de frame a X Hz, Ángulo cadera-rodilla_p1, Ángulo rodilla-tobillo_p1, Ángulo cadera-tobillo_p1, Ángulo cadera-rodilla_p2, Ángulo rodilla-tobillo_p2, Ángulo cadera-tobillo_p2 
    self.framerate=tracking_fps
    self.tracking=tracking_array

    #Crea una lista vacía para guardar frames de simulación (60 Hz) y otras opciones
    self.height=480
    self.width=640
    self.frames=[]
    self.renderer = mujoco.Renderer(self.model, self.height, self.width)

    #Inicializa el primer objetivo
    self.obj_frame=1
    self.objective_pos=-self.tracking[self.obj_frame-1][7]


  #Función donde se accionan los controladores, según la acción y los límites
  def actuate_model(self, action):
    #Añade a los controladores los ángulos del vector de acciones
    self.data.actuator("motor1_cadera_pierna1").ctrl+=np.clip(action[0],-self.max_step_motors,self.max_step_motors)
    self.data.actuator("motor2_cadera_pierna1").ctrl+=np.clip(action[1],-self.max_step_motors,self.max_step_motors)
    self.data.actuator("motor_rodilla_pierna1").ctrl+=np.clip(action[2],-self.max_step_motors,self.max_step_motors)
    self.data.actuator("motor_tobillo_pierna1").ctrl+=np.clip(action[3],-self.max_step_motors,self.max_step_motors)
    self.data.actuator("motor1_cadera_pierna2").ctrl+=np.clip(action[4],-self.max_step_motors,self.max_step_motors)
    self.data.actuator("motor2_cadera_pierna2").ctrl+=np.clip(action[5],-self.max_step_motors,self.max_step_motors)
    self.data.actuator("motor_rodilla_pierna2").ctrl+=np.clip(action[6],-self.max_step_motors,self.max_step_motors)
    self.data.actuator("motor_tobillo_pierna2").ctrl+=np.clip(action[7],-self.max_step_motors,self.max_step_motors)

    #self.data.actuator("arnes1").ctrl=-self.objective_pos+0.05
    #self.data.actuator("arnes2").ctrl=-self.objective_pos+0.05
    #self.data.actuator("arnes3").ctrl=-self.objective_pos+0.05
    
    #Usa clip para aplicar los límites superiores e inferiores
    self.data.actuator("motor1_cadera_pierna1").ctrl=np.clip(self.data.actuator("motor1_cadera_pierna1").ctrl,-0.95,0.698)
    self.data.actuator("motor2_cadera_pierna1").ctrl=np.clip(self.data.actuator("motor2_cadera_pierna1").ctrl,-0.698,0.95)
    self.data.actuator("motor_rodilla_pierna1").ctrl=np.clip(self.data.actuator("motor_rodilla_pierna1").ctrl,0,1.48)
    self.data.actuator("motor_tobillo_pierna1").ctrl=np.clip(self.data.actuator("motor_tobillo_pierna1").ctrl,-0.31,0.61)
    self.data.actuator("motor1_cadera_pierna2").ctrl=np.clip(self.data.actuator("motor1_cadera_pierna2").ctrl,-0.95,0.698)
    self.data.actuator("motor2_cadera_pierna2").ctrl=np.clip(self.data.actuator("motor2_cadera_pierna2").ctrl,-0.698,0.95)
    self.data.actuator("motor_rodilla_pierna2").ctrl=np.clip(self.data.actuator("motor_rodilla_pierna2").ctrl,0,1.48)
    self.data.actuator("motor_tobillo_pierna2").ctrl=np.clip(self.data.actuator("motor_tobillo_pierna2").ctrl,-0.31,0.61)

    #self.data.actuator("arnes1").ctrl=np.clip(self.data.actuator("arnes1").ctrl,-1,2)
    #self.data.actuator("arnes2").ctrl=np.clip(self.data.actuator("arnes2").ctrl,-1,2)
    #self.data.actuator("arnes3").ctrl=np.clip(self.data.actuator("arnes3").ctrl,-1,2)
    


  def step(self,action):

    #Ejecuta la acción y se cambian los controladores en la simulación
    self.actuate_model(action)
    #Da el paso de simulación
    mujoco.mj_step(self.model, self.data)
    #Lee las nuevas posiciones (estado)
    self.state=np.concatenate((self.data.geom('TrackerCadera_pierna1_geom').xpos,
                              self.data.geom('TrackerRodilla_pierna1_geom').xpos,
                              self.data.geom('TrackerTobillo_pierna1_geom').xpos,
                              self.data.geom('TrackerCadera_pierna2_geom').xpos,
                              self.data.geom('TrackerRodilla_pierna2_geom').xpos,
                              self.data.geom('TrackerTobillo_pierna2_geom').xpos,
                              np.array([self.data.time])))

    #Calcula una reward, tiempo de simulación y objetivo de tracking de movimiento en frame específico
    #PENDIENTE: implementar cálculo de cuaterniones
    #PENDIENTE: implementar velocidad, end effector, center of mass
    #calcula los ángulos entre los puntos de tracking
    # 0  Cadera pierna 1 x
    # 1  Cadera pierna 1 y
    # 2  Cadera pierna 1 z
    # 3  Rodilla pierna 1 x
    # 4  Rodilla pierna 1 y
    # 5  Rodilla pierna 1 z
    # 6  Tobillo pierna 1 x
    # 7  Tobillo pierna 1 y
    # 8  Tobillo pierna 1 z
    # 9  Cadera pierna 2 x
    # 10 Cadera pierna 2 y
    # 11 Cadera pierna 2 z
    # 12 Rodilla pierna 2 x
    # 13 Rodilla pierna 2 y
    # 14 Rodilla pierna 2 z
    # 15 Tobillo pierna 2 x
    # 16 Tobillo pierna 2 y
    # 17 Tobillo pierna 2 z
    #*se multiplica por menos porque el eje coordenado y en MuJoCo está invertido con respecto al tracking
    current_angle_cadera_rodilla_pierna1=np.arctan((-(self.state[4]-self.state[1]))/np.abs((self.state[5]-self.state[2])))
    current_angle_rodilla_tobillo_pierna1=np.arctan((-(self.state[7]-self.state[4]))/np.abs((self.state[8]-self.state[5])))
    current_angle_cadera_tobillo_pierna1=np.arctan((-(self.state[7]-self.state[1]))/np.abs((self.state[8]-self.state[2])))
    current_angle_cadera_rodilla_pierna2=np.arctan((-(self.state[13]-self.state[10]))/np.abs((self.state[14]-self.state[11])))
    current_angle_rodilla_tobillo_pierna2=np.arctan((-(self.state[16]-self.state[13]))/np.abs((self.state[17]-self.state[14])))
    current_angle_cadera_tobillo_pierna2=np.arctan((-(self.state[16]-self.state[10]))/np.abs((self.state[17]-self.state[11])))
    current_pos=self.state[1]
    #revisa el tiempo actual de simulación, contraste con los objetivos de la captura de movimiento, y revisa los ángulos objetivo
    if(self.data.time > self.obj_frame/self.framerate):
      self.obj_frame+=1
    objective_angle_cadera_rodilla_pierna1=self.tracking[self.obj_frame-1][1]
    objective_angle_rodilla_tobillo_pierna1=self.tracking[self.obj_frame-1][2]
    objective_angle_cadera_tobillo_pierna1=self.tracking[self.obj_frame-1][3]
    objective_angle_cadera_rodilla_pierna2=self.tracking[self.obj_frame-1][4]
    objective_angle_rodilla_tobillo_pierna2=self.tracking[self.obj_frame-1][5]
    objective_angle_cadera_tobillo_pierna2=self.tracking[self.obj_frame-1][6]
    self.objective_pos=-self.tracking[self.obj_frame-1][7]
    #calcula reward
    rp=np.exp(-2*((objective_angle_cadera_rodilla_pierna1-current_angle_cadera_rodilla_pierna1)**2+(objective_angle_rodilla_tobillo_pierna1-current_angle_rodilla_tobillo_pierna1)**2+(objective_angle_cadera_rodilla_pierna2-current_angle_cadera_rodilla_pierna2)**2+(objective_angle_rodilla_tobillo_pierna2-current_angle_rodilla_tobillo_pierna2)**2))
    rv=0
    re=np.exp(-40*((objective_angle_cadera_tobillo_pierna1-current_angle_cadera_tobillo_pierna1)**2+(objective_angle_cadera_tobillo_pierna2-current_angle_cadera_tobillo_pierna2)**2))
    rc=np.exp(-10*((self.objective_pos-current_pos)**2))
    reward=0.65*rp+0.1*rv+0.15*re+0.1*rc*0

    #Condiciones de finalización
    #si pierde el equilibrio y se cae
    done=False
    if(self.state[2]<-0.15 or self.state[11]<-0.15):
      done=True
      reward=-80
    if(self.data.time >= self.final_simulation_time):
      done=True

    #Complementar
    info={
      "reward": reward,
      "state": self.state,
      "objective": self.tracking[self.obj_frame-1],
      "time": self.data.time
    }

    #Retornos
    return self.state, reward, done, info


  def render(self):
    #después de la función step(action) se llama a esta función, así que se comentan algunas líneas
    if len(self.frames) < self.data.time * self.framerate:
      self.renderer.update_scene(self.data, camera='camara_rodilla')
      pixels = self.renderer.render()
      self.frames.append(pixels)

  def reset(self):
    self.frames=[] #borra los frames
    self.obj_frame=1 #reinicia el objetivo de tracking
    mujoco.mj_resetData(self.model, self.data) #reinicia la simulación
    self.state=np.concatenate((self.data.geom('TrackerCadera_pierna1_geom').xpos,
                              self.data.geom('TrackerRodilla_pierna1_geom').xpos,
                              self.data.geom('TrackerTobillo_pierna1_geom').xpos,
                              self.data.geom('TrackerCadera_pierna2_geom').xpos,
                              self.data.geom('TrackerRodilla_pierna2_geom').xpos,
                              self.data.geom('TrackerTobillo_pierna2_geom').xpos,
                              np.array([self.data.time]))) #cambia el estado tras el reinicio
    self.objective_pos=-self.tracking[self.obj_frame-1][7]
    return self.state


In [ ]:
eval_env2=RobotBipedoEnv2(xml_text=xml, tracking_array=array_datos, tiempo_max_sim=2)
state=eval_env2.reset()
new_state=state
done=False
recompensa_acumulada=0
pasos=0

while not done:
  state=new_state
  pasos+=1
  action, _states = model.predict(state, deterministic=True)
  #print(action)
  new_state,reward,done,info=eval_env2.step(action)
  recompensa_acumulada+=reward
  eval_env2.render()
  print()

print("Recompensa total {} en {} pasos de tiempo.".format(recompensa_acumulada,pasos))
media.show_video(eval_env2.frames, fps=eval_env2.framerate)

0.004540999773179909
0.7336186404695142
0.004540999773179909
0.7335942675024838
0.004540999773179909
0.733612484986293
0.004540999773179909
0.733720722777913
0.004540999773179909
0.7339478241000564
0.004540999773179909
0.7343069778170389
0.004540999773179909
0.7347946973018258
0.004540999773179909
0.7353932718177625
0.004540999773179909
0.7360737289138373
0.004540999773179909
0.7367980293260843
0.004540999773179909
0.737515215838831
0.004540999773179909
0.7381477428757632
0.004540999773179909
0.7384935125026547
0.004540999773179909
0.7385104882392106
0.004540999773179909
0.7384239140956992
0.004540999773179909
0.738357507988795
0.004540999773179909
0.7221815984861012
0.004540999773179909
0.7221799786939379
0.004540999773179909
0.722203187196313
0.004540999773179909
0.7222277728624815
0.004540999773179909
0.722232126586378
0.004540999773179909
0.7222108290771799
0.004540999773179909
0.7221811449622706
0.004540999773179909
0.722158690807664
0.004540999773179909
0.7221493187570237
0.00454

This browser does not support the video tag.
